In [15]:
import pandas as pd
import numpy as np

In [16]:
df= pd.read_csv(r'C:\Users\Marcelo\UNIFEI\github\AI-UNIFEI\Base de dados Planta de Processo\base2024_classificacao.csv')
df.head()

,dispositivo_1,dispositivo_2,dispositivo_3,dispositivo_4,dispositivo_5,dispositivo_6,dispositivo_7,dispositivo_8,dispositivo_9,dispositivo_10,...,dispositivo_42,dispositivo_43,dispositivo_44,dispositivo_45,dispositivo_46,dispositivo_47,dispositivo_48,dispositivo_49,dispositivo_50,falha
0,73.18,61.70,44.79,34.70,64.35,31.37,71.95,46.84,45.40,57.63,...,57.50,49.11,35.51,49.83,35.35,56.37,56.21,50.41,42.17,0
1,48.70,36.58,42.64,51.02,66.17,43.68,51.84,57.06,40.92,33.10,...,42.58,45.03,55.41,56.54,34.13,50.11,49.88,49.82,69.11,0
2,45.65,69.17,48.58,34.39,42.41,41.61,59.15,55.03,59.03,59.72,...,74.03,48.05,39.78,58.47,63.05,54.80,68.53,45.07,71.07,0
3,63.11,49.81,38.17,59.98,61.59,59.39,48.50,55.62,52.20,30.47,...,43.08,47.89,32.30,66.46,54.78,60.01,21.40,53.12,50.01,0
4,28.41,38.22,43.15,39.12,58.32,71.58,36.61,45.84,35.68,45.38,...,58.20,55.04,36.48,52.88,54.85,66.86,50.58,58.64,53.66,0


In [17]:
from sklearn.model_selection import train_test_split

X = df.drop('falha', axis=1).values
y= df['falha'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.85, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((25920, 50), (146880, 50), (25920,), (146880,))

In [18]:
#Incorporando um pipeline no aprendizado de máquina
from sklearn.pipeline import Pipeline

#Incorporando pré-processamento na base de dados e redução de dimensionalidade da base de dados
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD

#Incorporando uma classficação cruzada para avaliar as técnicas utilizadas
from sklearn.model_selection import cross_validate

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from tqdm import tqdm

prepros = [
    None,
    StandardScaler(),
    MinMaxScaler(),
]

redutores = [
    None,
    PCA(random_state=42),
    TruncatedSVD(random_state=42),
]

aprendizados = [
    RandomForestClassifier(random_state=42),
    DecisionTreeClassifier(random_state=42),
    SVC(random_state=42),
    LogisticRegression(random_state=42),
    MLPClassifier(random_state=42),
    DummyClassifier(random_state=42),
]

resultados = []

for pp, red, ap, in tqdm([(pp, red,ap) for pp in prepros for red in redutores for ap in aprendizados]):
    steps=[]
    if pp is not None:
        steps.append((pp.__class__.__name__, pp))
    
    if red is not None:
        steps.append((red.__class__.__name__, red))
    
    steps.append((ap.__class__.__name__, ap))

    pipe = Pipeline(steps=steps)
    cv= cross_validate(pipe, X_train, y_train, cv=5, scoring=['accuracy', 'f1'], n_jobs=-1)
    resultados.append({
        'prepros': pp.__class__.__name__,
        'redutor': red.__class__.__name__,
        'aprendizado': ap.__class__.__name__,
        'acuracia': np.mean(cv['test_accuracy']),
        'f1': cv['test_f1'].mean(),
    })

df_resultados = pd.DataFrame(resultados)
#Ordenamento decrescente dos melhores modelos avaliados pelas métrica
df_resultados.sort_values('f1', ascending=False)

  0%|          | 0/54 [00:00<?, ?it/s]

100%|██████████| 54/54 [33:35<00:00, 37.32s/it] 


,prepros,redutor,aprendizado,acuracia,f1
46,MinMaxScaler,PCA,MLPClassifier,0.900193,0.900193
18,StandardScaler,NoneType,RandomForestClassifier,0.890046,0.887606
0,NoneType,NoneType,RandomForestClassifier,0.889931,0.887511
36,MinMaxScaler,NoneType,RandomForestClassifier,0.889892,0.887467
40,MinMaxScaler,NoneType,MLPClassifier,0.879668,0.881423
28,StandardScaler,PCA,MLPClassifier,0.873611,0.873866
22,StandardScaler,NoneType,MLPClassifier,0.871181,0.871854
20,StandardScaler,NoneType,SVC,0.863580,0.859726
26,StandardScaler,PCA,SVC,0.863580,0.859726
8,NoneType,PCA,SVC,0.860725,0.856575
